# **CHAPTER_8**

## **8.1 계층적 색인**

계층적 색인은 축에 대해 다중 색인 단계를 지정할 수 있도록 하며 데이터의 부분집합을 부분적 색인으로 접근하는 것이 가능하다. 계층적 인덱스의 원칙은 항상 가장 상위 계층에 해당하는 맨 바깥쪽 인덱스부터 차례대로 인덱싱하는 것이다. 

In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [ ]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [ ]:
# 하위 계층의 객체 선택, 각 계층의 하위 계층이 2인 값만을 선택한다. 

data.loc[:, 2]

a    1.007189
c    0.886429
d   -2.001637
dtype: float64

In [ ]:
# unstack 메서드를 이용해 데이터를 새롭게 배열할 수 있다. 기존의 data에서 내용은 동일하지만 모양이 다른 프레임이 생성된다. 이후 stack 메서드를 통해 복구할 수 있다. 

data.unstack()

,1,2,3
a,1.246435,1.007189,-1.296221
b,0.274992,NaN,0.228913
c,1.352917,0.886429,NaN
d,NaN,-2.001637,-0.371843


**두 축 모두 계층적 색인을 가진 경우**

In [ ]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),  'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'], 'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [ ]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)), index = [['a', 'a', 'b', 'b'], [1, 2, 1, 2]], columns=[['Ohio','Ohio','Colorado'],['Green','Red', 'Green']])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [ ]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

### **8.1.1 계층의 순서를 바꾸고 정렬하기**

계층적 색인에서 계층의 순서를 바꾸거나 지정된 계층에 따라 데이터를 정렬해야하는 경우 swaplevel을 통해 뒤바뀐 객체를 반환할 수 있다. 다만 이 경우 데이터는 변경되지 않는다. 

In [ ]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [ ]:
frame.swaplevel(0,1).sort_index(level = 0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### **8.1.2 계층별 요약 통계**

In [ ]:
frame.sum(level = 'color', axis = 1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### **8.1.3 DataFrame의 컬럼 사용하기**

행의 색인을 컬럼으로 사용하거나 행을 선택하기 위한 색인으로 하나 이상의 컬럼을 사용하고자하는 경우가 있을 때 아래와 같은 방법을 사용한다. 

In [ ]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1), 'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'], 'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [ ]:
frame2 = frame.set_index(['c', 'd'])

## **8.2 데이터 합치기**

### **8.2.1 데이터베이스 스타일로 DataFrame 합치기**

In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [ ]:
# merge 함수를 이용하여 두 데이터프레임에 대해 중복된 컬럼 이름을 키로 사용할 수 있다. 

pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [ ]:
# 키 이름이 다를 경우 각각의 병합 index를 지정하여 사용할 수 있다. 겹치는 값 이외에 외부적인 값 또한 출력되게 하기 위해서는 옵션을 outer로 지정하여 모든 로우를 포함하는 결과를 반환할 수 있게 한다. 

df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


### **8.2.2 색인 병합하기**

병합하려는 키가 데이터프레임의 색인인 경우 x_index = True 옵션을 사용하여 해당 색인을 병합키로 사용할 수 있다. 

In [ ]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [ ]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio','Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio','Ohio', 'Ohio'], [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [ ]:
pd.merge(lefth, righth, left_on = ['key1', 'key2'], right_index = True, how = 'outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [ ]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [ ]:
# join 메서드는 칼럼이 겹치지 않으며 완전히 같거나 유사한 색인 구조를 가진 여러 데이터프레임 객체를 병합할 수 있다. 

In [ ]:
left1.join(right1, on = 'key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


### **8.2.3 축 따라 이어붙이기**

In [ ]:
arr =  np.arange(12).reshape((3,4))

In [ ]:
np.concatenate([arr, arr], axis = 1)

# pandas에서는 concat 함수의 옵션을 조정해 교집합 등을 구할 수 있으며 축 기준으로 병합 가능하다. 

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [ ]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [ ]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])

In [ ]:
# 리스트 대신 객체의 사전을 넘길 경우 사전의 키가 keys 옵션으로 사용된다. 

pd.concat([df1, df2], axis = 1, keys = ['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

### **8.2.4 겹치는 데이터 합치기**

색인이 일부 겹치거나 전체가 겹치는 경우 병합이나 이어붙이기가 불가능하다. 이때는 series 객체의 combine_first 메서드를 사용해 정렬할 수 있다. 

In [ ]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan

In [ ]:
np.where(pd.isnull(a), b, a) # b를 기준으로 데이터를 채운다.  

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [ ]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

In [ ]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## **8.3 재형성과 피벗**

### **8.3.1 계층적 색인으로 재형성하기**

- stack: 데이터의 칼럼을 로우로 피벗(회전) 시킨다. 
- unstack: 로우를 칼럼으로 피벗시킨다. 

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))

In [ ]:
# 칼럼이 로우로 피벗되어 Series 객체를 반환한다. 또한 누락된 데이터를 자동으로 걸러내기에 연산을 쉽게 복구할 수 있다. 

data.stack()

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [ ]:
result = data.stack()

# 데이터를 unstack하면 레벨은 결과에서 가장 낮은 단계가 된다. 

result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [ ]:
df = pd.DataFrame({'left': result, 'right': result + 5}, columns=pd.Index(['left', 'right'], name='side'))

In [ ]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### **8.3.2 긴 형식에서 넓은 형식으로 피벗하기** 

데이터 베이스나 csv 파일에서 여러 개의 시계열 데이터를 저장하는 방법은 시간 순서대로 나열하는 것이다. 긴 형식의 경우 여러 시계열이나 둘 이상의 키를 가지고 있는 다른 관측데이터에서 사용한다. 각 로우는 단일 관측치를 나타낸다. 

MySQL 같은 관계형 데이터베이스의 경우 고정된 스키마에 데이터를 저장하여 데이터를 편집한다. 컬럼은 기본키가 되어 관계 무결성의 성질을 만족하게하며 조인 연산 등을 가능하게 한다. 길이가 긴 형식의 경우 작업에 용이하지 않을 수 있어서 색인된 컬럼을 선호하는 경우도 존재한다. 

In [ ]:
data = pd.read_csv('/content/macrodata.csv')

In [ ]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})

In [ ]:
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [ ]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [ ]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,0.022074
1,1959-03-31 23:59:59.999999999,infl,0.000,-1.369179
2,1959-03-31 23:59:59.999999999,unemp,5.800,0.887204
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,0.014331
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.074155
5,1959-06-30 23:59:59.999999999,unemp,5.100,-0.048565
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,1.235021
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.433295
8,1959-09-30 23:59:59.999999999,unemp,5.300,1.391035
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,0.820211


In [ ]:
ldata['value2'][:5]

0    0.022074
1   -1.369179
2    0.887204
3    0.014331
4   -0.074155
Name: value2, dtype: float64

In [ ]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value            ...    value2          
item                           infl   realgdp  ...   realgdp     unemp
date                                           ...                    
1959-03-31 23:59:59.999999999  0.00  2710.349  ...  0.022074  0.887204
1959-06-30 23:59:59.999999999  2.34  2778.801  ...  0.014331 -0.048565
1959-09-30 23:59:59.999999999  2.74  2775.488  ...  1.235021  1.391035
1959-12-31 23:59:59.999999999  0.27  2785.204  ...  0.820211  0.302271
1960-03-31 23:59:59.999999999  2.31  2847.699  ...  0.543980 -1.266383
1960-06-30 23:59:59.999999999  0.14  2834.390  ...  0.937250 -1.593952
1960-09-30 23:59:59.999999999  2.70  2839.022  ... -0.375498  0.794336

[7 rows x 6 columns]

### **8.3.3 넓은 형식에서 긴 형식으로 피벗하기**

In [ ]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [ ]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [ ]:
# pivot을 이용해서 원래 모양으로 되돌린다. 또한 reset_index를 이용해서 다시 컬럼으로 돌려놓을 수 있다. 

reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [ ]:
# 데이터 값으로 사용할 컬럼들의 집합 지정도 할 수 있다. 
pd.melt(df, id_vars = ['key'], value_vars=['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
